### 비정형 데이터 융합형 기업 예측 모델
* 뉴스나 전문가 의견및 여론 기반한 비정형 데이터 수집
* 주가 등 정형 데이터 수집
* 기업의 주가 영향 요소 예측등을 통한
* 기업의 위험관리및 투자예측 분석

설립일:1955년 12월 9일 (65주년)[2]
주소
서울특별시 서초구 신반포로 194 (반포동)[3]
업종명
백화점
상장유무
상장기업
기업규모
대기업
상장시장
유가증권시장 (1985년 ~ 현재)
대표이사
차정호
종목코드
004170
자본금
492억 2천만원 (2019)
매출액
1조 5,576억원 (2019)
영업이익
2,220억 9천만원 (2019)
당기순이익
7,059억 5천만원 (2019)
직원 수
2,745명 (2020.06.30)

### 분석요소
* 뉴스 :
* SNS  :
* 댓글 :
* 재무재표: 
* 기업분석 지표:

* 제품 매출 현황

* 시가 :
* 종가 :
* 고가 :
* 저가 :
* 거래량:

### 네이버 뉴스 수집
* 신세계
* url = https://search.naver.com/search.naver?query=%EC%8B%A0%EC%84%B8%EA%B3%84&where=news&ie=utf8&sm=nws_hty

In [2]:
url='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'.format('신세계',11)
url

'https://search.naver.com/search.naver?&where=news&query=신세계&start=11'

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [3]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [4]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [6]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [7]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [8]:
def insMain(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into nnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [9]:
def getPage(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    tgts=htmls.select('ul.list_news > li')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [10]:
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}&pd=3&ds=2019.01.01&de=2019.12.31'
for i in range(100):
    url=urlsrc.format('신세계',i*10+1)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=신세계&start=1&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=1&pd=3&ds=2019.01.01&de=2019.12.31
..................................https://search.naver.com/search.naver?&where=news&query=신세계&start=11&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=11&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계 프리미엄 아울렛   NEW YEAR FESTA  개최
이미 존재 합니다.:신세계사이먼 프리미엄 아울렛   뉴 이어 페스타  개최
이미 존재 합니다.:신세계사이먼 프리미엄 아울렛  코트 패딩 최대 80  할인 판매
이미 존재 합니다.:신세계사이먼 프리미엄 아울렛   NEW YEAR FESTA  개최
이미 존재 합니다.:신세계사이먼 프리미엄 아울렛   NEW YEAR FESTA  개최 새해 득템 
..............https://search.naver.com/search.naver?&where=news&query=신세계&start=21&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=21&pd=3&ds=2019.01.01&de=2019.12.31
....................https://search.naver.com/search.naver?&where=news&query=신세계&start=31&pd=3&ds=2019.01.

https://search.naver.com/search.naver?&where=news&query=신세계&start=161&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계푸드 올반X요리연구가 문성실  멘보샤 핫도그  출시
이미 존재 합니다.:신세계푸드   에어프라이어 전용HMR  2종 출시
이미 존재 합니다.:신세계푸드  요리연구가와 협업  에어프라이어 전용 HMR  2종 출시
이미 존재 합니다.:신세계푸드  요리연구가 문성실 협업  에어프라이어 전용HMR  출시
.이미 존재 합니다.: 더벨 신세계푸드   세린식품 스무디킹  엇갈린 인수 성적표
이미 존재 합니다.:12월 30일 신세계 I C 주식현황 108000원 2 86 
이미 존재 합니다.:신세계백화점  커피 찌꺼기로 키운 농산물 판매
이미 존재 합니다.:신세계百  지역 농가에 커피 찌꺼기로 만든 비료 기부
이미 존재 합니다.:신세계百  커피 찌꺼기로 만든 비료 중소농가 기부
이미 존재 합니다.:신세계百  커피 찌꺼기로  천연비료  만들어 농가에 기부한다
.이미 존재 합니다.:씨엔블루 정용화  신세계조선호텔 홍보대사
이미 존재 합니다.:정용화  신세계조선호텔 홍보모델 위촉
이미 존재 합니다.: 백화점과일반상점증시시황  백화점과일반상점 업종리드 종목  신세계 2 67 상   
이미 존재 합니다.: 뉴스락 특별기획 10대 그룹은 변신 중 ㊦   한화 GS 현대중공업 신세계    
.....이미 존재 합니다.:신세계百  다음달 6일부터 설 선물세트 본판매 시작
이미 존재 합니다.:신세계백화점  1월 6일부터 설 본판매 시작  프리미엄 제품 강화 
이미 존재 합니다.:신세계백화점  1월 6일부터 설 본판매 시작
이미 존재 합니다.:신세계百  설 선물세트 본판매 프리미엄부터 가성비까지 36만 세트 준비
.이미 존재 합니다.: 더벨  비전 2020 점검 신세계인터  10년만에 상전벽해 화장품이 견인
https://search.naver.com/search.naver?&where

https://search.naver.com/search.naver?&where=news&query=신세계&start=241&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계백화점  1월6일부터 설 본판매 시작
이미 존재 합니다.:신세계백화점  내년 1월6일부터 설 선물세트 본판매
이미 존재 합니다.:신세계百  내년 1월6일부터 설 선물 본판매 시작
이미 존재 합니다.:신세계百  설 선물세트 본판매  인기맛집 메뉴  1인가구 겨냥 
이미 존재 합니다.:신세계 바이레도  튤립마니아  핸드케어 라인 출시
.이미 존재 합니다.:신세계푸드  소아암 환아에 사랑의 헌혈증 1004매 기부
이미 존재 합니다.:신세계푸드  소아암 환아에 사랑의 헌혈증 1004매 기부
이미 존재 합니다.:소아암 아동 돕는 신세계푸드  헌혈증 1004장 후원금 전달
..이미 존재 합니다.:양과 질 모두 잡는다 신세계  온라인 배송 시스템 또 강화
이미 존재 합니다.:신세계아이앤씨  사이버보안 시장 진출
이미 존재 합니다.:신세계아이앤씨  클라우드 AI 기반 차세대 정보보안 시장 진출
...이미 존재 합니다.:신세계인터  佛가란시아 국내 론칭 더마코스메틱 공략
이미 존재 합니다.:신세계인터  화장품 사업 확대 더마 코스메틱 시장 진출
이미 존재 합니다.:정유경의 신세계인터내셔날  화장품 사업서 거침없는 성장세
이미 존재 합니다.:신세계인터  佛 가란시아 국내 론칭 더마 코스메틱 시장 진출
.이미 존재 합니다.:신세계푸드  요리연구가 문성실과 협업  에어프라이어 전용HMR  2종 출시
이미 존재 합니다.:신세계푸드  요리연구가 문성실과  에어프라이어 전용 간편식  출시
이미 존재 합니다.:신세계푸드  요리연구가 문성실 협업  에어프라이어 전용HMR  2종 출시
이미 존재 합니다.:신세계푸드  요리연구가 문성실과 협업  에어프라이어 전용HMR  2종 출시
.이미 존재 합니다.:신세계  제주소주   국내 최초 포장재  최우수등급  페트 제품 출시
이미 존재 합니다.

https://search.naver.com/search.naver?&where=news&query=신세계&start=311&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.: 백화점과일반상점 업종 오후 시황  상승률 상위 종목  신세계  3 02  롯데쇼   
이미 존재 합니다.:신세계아이앤씨  클라우드 AI 기반 차세대 정보보안 시장 진출
이미 존재 합니다.:신세계I C  AI 기반 통합 보안관제  시스가드  출시
이미 존재 합니다.:신세계I C  정보보안 시장 출사표 보안 서비스  시스가드  출시
이미 존재 합니다.:클라우드 AI 기반 정보보안서비스  신세계아이앤씨  시스가드  선봬
이미 존재 합니다.:신세계아이앤씨  클라우드 AI 기술 기반 차세대 정보보안 시장 진출
이미 존재 합니다.:신세계 오너家 정유경  SI 지분 17개월만에 또 매각 왜 
이미 존재 합니다.:신세계 정유경  신세계인터 주식 30만주 매각
이미 존재 합니다.:신세계인터 30만주 팔아 664억 확보한 정유경 사장
.이미 존재 합니다.:신세계인터내셔날  프랑스 더마화장품  가란시아  들여온다
이미 존재 합니다.:신세계인터내셔날  더마코스메틱 시장 공략
...이미 존재 합니다.:신세계 롯데 심기일전  유통공룡 2020년  온라인 혈투  미리보기
이미 존재 합니다.: 부고  조창현 신세계 사이먼 대표 씨 장모상
이미 존재 합니다.:조창현 신세계사이먼 대표 빙모상
이미 존재 합니다.: 부고  조창현 신세계 사이먼 대표 씨 장모상
이미 존재 합니다.:신세계百  설 선물세트 본판매  인기맛집 메뉴  1인가구 겨냥 
.이미 존재 합니다.:신세계조선호텔  부산 웨스틴조선 리뉴얼 전면 중단  잠정 보류
이미 존재 합니다.: 어린이 고객 모십니다   신세계百   영어 놀이학교  오픈
이미 존재 합니다.:신세계 영등포점  회원제 영어 키즈클럽 선봬 키즈 콘텐츠 강화
이미 존재 합니다.:키즈시설 들였더니 매출  쑥  신세계百  영등포점에  영어 키즈클럽  오   
이미 존재 

https://search.naver.com/search.naver?&where=news&query=신세계&start=401&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계백화점 JW메리어트  산타와 함께하는  케이크 팝업  행사
이미 존재 합니다.: 호텔 케이크부터 산타 마술쇼까지    신세계百  크리스마스 특별 이벤트 진행
이미 존재 합니다.:신세계百  크리스마스 이벤트 JW메리어트 케이크 팝업 운영
이미 존재 합니다.:신세계백화점  산타 만나고 호텔 케이크 사세요 
.이미 존재 합니다.: 사회공헌  신세계푸드  소아암 환아에 사랑의 헌혈증 1004매 기증
.이미 존재 합니다.:신세계 정유경  신세계인터 주식 30만주 매각
이미 존재 합니다.:신세계인터 30만주 팔아 664억 확보한 정유경 사장
이미 존재 합니다.:정유경 신세계 총괄사장  신세계인터내셔날 지분 대량 매도
.....이미 존재 합니다.:패션섬유 상장기업 브랜드파워 1위 휠라코리아  2위 신세계인터내셔날
..이미 존재 합니다.: 솔로이코노미 동향  신세계푸드 에어프라이어 전용 간편식 식재료 새벽 배송   
https://search.naver.com/search.naver?&where=news&query=신세계&start=411&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=411&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계 제주소주   최우수등급 포장재  적용한 제품 선보여
이미 존재 합니다.:신세계 제주소주  푸른밤   최우수등급 친환경 페트 제품 선보여
이미 존재 합니다.: 솔로이코노미 동향  신세계푸드 에어프라이어 전용 간편식 식재료 새벽 배송   
이미 존재 합니다.:정용화 신세계조선호텔 홍보모델 발탁  한류 마케팅 참여
..이미 존재 합니다.:SSG닷컴  온라인스토어  네오003

https://search.naver.com/search.naver?&where=news&query=신세계&start=501&pd=3&ds=2019.01.01&de=2019.12.31
..이미 존재 합니다.:광주신세계  희망산타원정대  발대 나눔활동 전개
이미 존재 합니다.:광주신세계  희망 산타 원정대  발대식 가져
.이미 존재 합니다.:신세계 004170   24일  0 88 변화 증권그래프 살펴보기
.....이미 존재 합니다.:12월 23일 신세계종목정보  0 17  변동
이미 존재 합니다.:SSG닷컴  온라인스토어  네오003  본격 가동    e커머스에 신세계 연다 
이미 존재 합니다.:신세계  온라인 심장  네오003  가동  새벽배송 깨운다
이미 존재 합니다.:신세계  쓱닷컴  세번째 물류센터 오픈  수도권 새벽배송 전쟁
이미 존재 합니다.:신세계면세점   KEEP IT GREEN  업사이클링 캠페인 전개
이미 존재 합니다.:신세계면세점   KEEP IT GREEN  업사이클링 캠페인 전개
이미 존재 합니다.:신세계인터내셔날  프랑스 약국 화장품  가란시아  론칭
이미 존재 합니다.:신세계인터내셔날  프랑스 브랜드  가란시아  론칭 더마 코스메틱 진출
이미 존재 합니다.:신세계TV쇼핑  신선 식품 PB  테이스트킹  론칭
이미 존재 합니다.:신세계TV쇼핑  가심비  식품PB  테이스트 킹  론칭
이미 존재 합니다.:신세계TV쇼핑  식품 PB  테이스트킹  선봬
.이미 존재 합니다.:뉴요커가 만든 한국식  토끼소주   신세계百서 단독 판매
이미 존재 합니다.: 포토  신세계百  미국에서 만든  토끼소주  맛보세요 
이미 존재 합니다.:신세계백화점  미국 뉴욕  토끼소주  단독 출시
https://search.naver.com/search.naver?&where=news&query=신세계&start=511&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news

https://search.naver.com/search.naver?&where=news&query=신세계&start=611&pd=3&ds=2019.01.01&de=2019.12.31
.이미 존재 합니다.:플라스틱 줄이기 운동 동참  신세계免   킵 잇 그린  업사이클링 캠페인 전개
.이미 존재 합니다.:신세계TV쇼핑  그루밍족 위한 남성 상품 강화 나선다
이미 존재 합니다.:신세계TV쇼핑  남성 상품 강화 나선다 그루밍족 공략
이미 존재 합니다.:신세계TV쇼핑  그루밍족 위한 남성 상품 강화 나선다
..이미 존재 합니다.: 포토  이번 설 선물은 신세계 상품권으로 하세요 
.이미 존재 합니다.:신세계백화점  뉴욕에서 만든  토끼소주  선봬
.이미 존재 합니다.:광주신세계  지역사랑  희망산타 원정대  발대
이미 존재 합니다.:광주신세계  소외계층 위한 희망 프로젝트  희망 산타 원정대  발대
이미 존재 합니다.:광주신세계   희망 산타 원정대  발대
..https://search.naver.com/search.naver?&where=news&query=신세계&start=621&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=621&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:이명희 신세계 회장 한남동집 전국 최고가 277억1000만원
...이미 존재 합니다.:현대 신세계百  내일부터 설 선물세트 예약판매
이미 존재 합니다.:신세계백화점  영어놀이학교 오픈
이미 존재 합니다.: 키즈 영어과외  백화점서 한다   신세계百  영어 키즈클럽  오픈
......이미 존재 합니다.:신세계TV쇼핑  남성 상품 강화 나선다 그루밍족 공략
이미 존재 합니다.:신세계TV쇼핑  그루밍족 위한 남성 상품 강화 나선다
......https://search.naver.com/search.naver?&where=news&qu

https://search.naver.com/search.naver?&where=news&query=신세계&start=731&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계百  미국인이 만든 전통소주  토끼소주  한정 판매
이미 존재 합니다.:뉴욕서 마시던  토끼소주   신세계백화점 한정 판매
이미 존재 합니다.:신세계百  브루클린서 태어난 전통주 단독 선봬
..이미 존재 합니다.: 인사  신세계그룹
이미 존재 합니다.: 인사  신세계그룹
이미 존재 합니다.: 인사 신세계그룹
.........이미 존재 합니다.:신세계푸드  조리사 꿈 키워주는  키다리 아저씨  2기 수료
이미 존재 합니다.:바캉스용품  한겨울에 산다 신세계百  여름상품 할인전
이미 존재 합니다.:신세계百  겨울 해외여행객 늘며 여름상품 판매 급증
이미 존재 합니다.:신세계百  겨울 해외여행족에 철없는 여름상품
.....https://search.naver.com/search.naver?&where=news&query=신세계&start=741&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=741&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.: 유통가 소식  신세계면세점  키엘 Kiehl s   명동점에서 팝업스토어 오픈 
.....이미 존재 합니다.:신세계 스타필드 레저시설  아쿠아필드  베트남 진출
.이미 존재 합니다.:신세계 대표에 차정호 신세계인터내셔날 대표
이미 존재 합니다.: 프로필 차정호 신세계 대표이사
이미 존재 합니다.: 新 시각   노련함  신세계  대표 맞트레이드
..이미 존재 합니다.:정유경 신세계 총괄사장  신세계인터 지분 4 2  매각   증여세 납부 목적
이미 존재 합니다.:정유경 총괄사장  신세계인터 지분 4 2  처분 1년새 930억 확보
이미 존재 합니다.:정유경 신세

https://search.naver.com/search.naver?&where=news&query=신세계&start=831&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계百  수영복 비치웨어 시즌오프 실시  최대 60  할인 
이미 존재 합니다.:신세계百  한겨울에 잘 팔리는 수영복  
이미 존재 합니다.:철없는 신세계백화점  한겨울 여름용품 할인
이미 존재 합니다.:신세계 I C  24일  0 46 변동 증권시세 및 매매가
이미 존재 합니다.:정유경 사장  900억원대 증여세 낼려고   신세계인터 30만주 664억에 팔아
이미 존재 합니다.:신세계 정유경 총괄사장  신세계인터 주식 30만주 매각 총 664억5300만원
이미 존재 합니다.: 공시  정유경 신세계 총괄사장  신세계인터내셔날 보유지분 4 2  매각
..이미 존재 합니다.:신세계百  16일부터  설 선물 예약 판매  실시
이미 존재 합니다.:벌써 신년 신세계百  설 선물세트 예약 판매 시작
..이미 존재 합니다.:대구신세계  오픈 3년만에 대구 대표 백화점으로 도약
이미 존재 합니다.:개점 3년 대구신세계 영업 호조에 동대구역 상권 활기
이미 존재 합니다.:개점 3년 대구 신세계 낙후된 동대구역 상권도 살렸다
..이미 존재 합니다.:신세계스타필드 아쿠아필드  베트남 진출한다
이미 존재 합니다.:신세계 스타필드 아쿠아필드  빈그룹 손잡고 베트남 간다
이미 존재 합니다.:신세계 스타필드  휴양 레저시설  아쿠아필드  베트남 진출
이미 존재 합니다.:신세계 스타필드 아쿠아필드  베트남 간다
이미 존재 합니다.:신세계그룹  백화점 임원인사  신세계인터내셔날과 대표 맞바뀐다
이미 존재 합니다.:장재영 차정호  자리 맞바꾼다 신세계  임원인사 단행
이미 존재 합니다.:신세계그룹 임원인사 단행  신세계 차정호 신임대표 임명
이미 존재 합니다.:신세계  차정호 대표 선임  장재영 대표와 맞트레이드
.https://search.naver.com/search.naver?&whe

https://search.naver.com/search.naver?&where=news&query=신세계&start=931&pd=3&ds=2019.01.01&de=2019.12.31
.이미 존재 합니다.:신세계푸드  온라인 베이커리 밀크앤허니  ㅋㅇㅋ  2종 출시
이미 존재 합니다.:신세계푸드  온라인 베이커리 공략 밀크앤허니  ㅋㅇㅋ  2종 출시
이미 존재 합니다.:신세계 베이커리  밀크앤허니    ㅋㅇㅋ 케익 2종 출시
..........이미 존재 합니다.: 정유경 남편  문성욱 부사장  신세계인터내셔날 신사업 이끈다
이미 존재 합니다.:정유경  신세계百 신임 대표로  차정호 사장  낙점
이미 존재 합니다.:신세계그룹 백화점부문  인사 단행  이길한 대표 연임한다
이미 존재 합니다.:신세계백화점에서 만난 뉴욕여행 인증 술  토끼소주 
이미 존재 합니다.:미국인이 만든 우리 전통 소주 신세계백화점에 등장
이미 존재 합니다.:신세계백화점에서 만난 뉴욕여행 인증 술  토끼소주 
이미 존재 합니다.:미국인이 만든 우리 전통 소주 신세계백화점에 등장
..https://search.naver.com/search.naver?&where=news&query=신세계&start=941&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=941&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계백화점에서 만난 뉴욕여행 인증 술  토끼소주 
이미 존재 합니다.:미국인이 만든 우리 전통 소주 신세계백화점에 등장
이미 존재 합니다.:미국인이 만든 우리 전통 소주 신세계백화점에 등장
.이미 존재 합니다.:정유경  신세계百 신임 대표로  차정호 사장  낙점
이미 존재 합니다.:신세계그룹 백화점부문  인사 단행  이길한 대표 연임한다
이미 존재 합니다.:울산 중구청장  신세계측과 면담  개발진행 논의
......이미 존재 합니다